In [1]:
import pandas as pd

data = pd.read_csv('../data/df_undersampling_multilabel.csv', index_col=0)  

In [2]:
data = data.reset_index()
data['case_type'].unique()

array(['CIVIL', 'COMMERCIAL', 'CRIMINAL', 'LABOR', 'POLITICAL',
       'REMEDIAL', 'TAX'], dtype=object)

In [3]:
data.groupby(by=["case_type"])['case_type'].count()

case_type
CIVIL         500
COMMERCIAL    174
CRIMINAL      500
LABOR         500
POLITICAL     389
REMEDIAL      500
TAX           500
Name: case_type, dtype: int64

In [4]:
data['is_civil'] = (data['case_type'] == 'CIVIL')
# data["is_civil"] = df["is_civil"].astype(int)

data['is_commercial'] = (data['case_type'] == 'COMMERCIAL')
# data["is_commercial"] = df["is_commercial"].astype(int)

data['is_criminal'] = (data['case_type'] == 'CRIMINAL')
# data["is_criminal"] = df["is_criminal"].astype(int)

data['is_labor'] = (data['case_type'] == 'LABOR')
# data["is_labor"] = df["is_labor"].astype(int)

data['is_political'] = (data['case_type'] == 'POLITICAL')
# data["is_political"] = df["is_political"].astype(int)

# data['is_remedial'] = (data['case_type'] == 'REMEDIAL')
# data["is_remedial"] = df["is_remedial"].astype(int)

data['is_tax'] = (data['case_type'] == 'TAX')
# data["is_tax"] = df["is_tax"].astype(int)

In [5]:
df = data[['content_clean', 'is_civil', 'is_commercial', 'is_criminal', 'is_labor', 'is_political',  'is_tax']]

In [6]:
df['labels'] = list(zip(df.is_civil.tolist(), df.is_commercial.tolist(), df.is_criminal.tolist(), df.is_labor.tolist(),  df.is_political.tolist(),  df.is_tax.tolist()))

<ipython-input-6-a90776066d10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = list(zip(df.is_civil.tolist(), df.is_commercial.tolist(), df.is_criminal.tolist(), df.is_labor.tolist(),  df.is_political.tolist(),  df.is_tax.tolist()))


In [7]:
df = df.rename(columns={"content_clean": 'text'})

In [8]:
df.head()

,text,is_civil,is_commercial,is_criminal,is_labor,is_political,is_tax,labels
0,Subject of this direct appeal to us on points ...,True,False,False,False,False,False,"(True, False, False, False, False, False)"
1,Petition for certiorari to annul and set aside...,True,False,False,False,False,False,"(True, False, False, False, False, False)"
2,This is a petition for review of the Decision[...,True,False,False,False,False,False,"(True, False, False, False, False, False)"
3,“The plaintiff must rely on the strength of [h...,True,False,False,False,False,False,"(True, False, False, False, False, False)"
4,The United Church of Christ in the Philippines...,True,False,False,False,False,False,"(True, False, False, False, False, False)"


In [9]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.2)

In [10]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('roberta', 'roberta-base', use_cuda=False, num_labels=6)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [11]:
model.train_model(train_df)

(307, 0.2639591163843385)

In [13]:
result_2, model_outputs_2, wrong_predictions_2 = model.eval_model(eval_df.head())

In [14]:
result_2

{'LRAP': 0.8666666666666668, 'eval_loss': 0.2524639666080475}